In [ ]:
import seaborn as sns
import shapely 
import geopandas as gp
from pathlib import Path
import masking as msk

### When to run this notebook

After creating raster masks in notebook 06 and converting them to vector masks using GDAL, you may want to tighten them somewhat to clean up edge effects, or combine with an ROI mask. This notebook can help. 

In [ ]:
projectdir = Path(r"Z:\fihyper\cwaigl\20200714_MKB")
inputdir = projectdir / r"02_intermediate/vectormasks_flightline" 
outputdir = projectdir / r"02_intermediate/vectormasks_cropped" 

In [ ]:
outputdir.exists()

True

In [ ]:
vectormasks = sorted(list(inputdir.glob(r"vnir_swir_*.shp")))
vectormasks

[WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_01_mask.shp'),
 WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_02_mask.shp'),
 WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_03_mask.shp'),
 WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_04_mask.shp'),
 WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_05_mask.shp'),
 WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_06_mask.shp'),
 WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_07_mask.shp'),
 WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_08_mask.shp'),
 WindowsPath('Z:/fihyper/cwaigl/20200714_MKB/02_intermediate/vectormasks_flightline/vnir_swir_09_mask.shp'),
 WindowsPath('Z:/fi

We generate a series of eroded polygons for use in masking. To smoothen out the edge, we buffer-and-erode first. 

In [ ]:
smoothbuff = 2
buffs = [0, 2, 5, 10, 20, 50]
crop = False
cropROI = Path(r"Z:\fiboreal\cwaigl\2021-HySpex_fuels\GIS\ROIs\CPC_ROI_CW.shp")
cropgdf = gp.read_file(cropROI)

In [ ]:
if crop:
    cropgdf

In [ ]:
for infile in vectormasks:
    testdata = gp.read_file(infile)
    for buff in buffs:
        testpoly = max(testdata.geometry, key=lambda a: a.area)
        outpoly = testpoly.buffer(smoothbuff).buffer((-1)*(smoothbuff + buff))
        outdata = gp.GeoSeries([outpoly], crs=testdata.crs)
        outfn = outputdir / f"{infile.stem}_buff{buff}.shp"
        print(f"writing {outfn}")
        outdata.to_file(outfn)
        if crop:
            cropoutdata = outdata.intersection(cropgdf)
            cropoutdata = gp.GeoSeries(max(cropoutdata, key=lambda a: a.area), crs=testdata.crs)
            outfn = outputdir / f"{infile.stem}_buff{buff}_crop.shp"
            print(f"writing {outfn}")
            cropoutdata.to_file(outfn)

writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_01_mask_buff0.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_01_mask_buff2.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_01_mask_buff5.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_01_mask_buff10.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_01_mask_buff20.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_01_mask_buff50.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_02_mask_buff0.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_02_mask_buff2.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cropped\vnir_swir_02_mask_buff5.shp
writing Z:\fihyper\cwaigl\20200714_MKB\02_intermediate\vectormasks_cro